In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

# Load the datasets
data_features = pd.read_csv("train_logs.csv")
data_scores = pd.read_csv("train_scores.csv")

# Merge the features and scores on 'id'
data = pd.merge(data_features, data_scores, on='id', how='left')

/Users/ayushlodha/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Create a mapping dictionary for the categories
action_category_mapping = {
    'Mouse Actions': ['Leftclick', 'Rightclick', 'Middleclick', 'Unknownclick'],
    'Keyboard Actions': ['Shift', 't', 's', 'n', 'y', 'q', '_', 'h', 'Space', 'Backspace', '.', 'F', 'g', 'M', 'l', '|', ',', 'I', '0', '1', '5', 'Enter', 'ArrowLeft', "'", ';', 'ArrowRight', '-', '?', 'Tab', '"', 'ArrowUp', 'ArrowDown', 'Control', 'c', 'v', '/', 'Delete', ':', 'z', '[', '$', '(', ')', '+', 'Home', 'End', '\\', 'Meta', '*', '&', 'x', '!', 'Insert', 'NumLock', '%', 'V', '>', 'Alt', 'a', '<', 'PageDown', ']', 'PageUp', 'Escape', 'AltGraph', 'OS', 'F', 'C', 'o', 'ä', 'F1', 'Pause','Unidentified', 'Cancel', '2', 'i', 'd', 'r', 'e', '`', 'm', '#', '~', 'T', 'A', 'b', 'S'],
    'Media Actions': ['AudioVolumeMute', 'MediaPlayPause', 'ContextMenu', 'AudioVolumeUp', 'AudioVolumeDown', 'MediaTrackPrevious', 'MediaTrackNext'],
    'Function Keys': ['F12', 'F3', 'F11', 'F10', 'F15', 'F2'],
    'Other Actions': ['Dead', '{', 'ModeChange', 'ScrollLock', '¿', 'Process', '}',  'Clear', 'â\x80\x93', '\x97', 'Ë\x86', '¡', '\x80', 'Â´', 'Å\x9f', '^']
}
# Create a new column 'action_category' in your DataFrame based on the mapping
data['down_event_action_category'] = data['down_event'].map({action: category for category, actions in action_category_mapping.items() for action in actions})
data['up_event_action_category'] = data['up_event'].map({action: category for category, actions in action_category_mapping.items() for action in actions})

In [3]:
data = data.drop(['down_event', 'up_event'], axis=1)

In [6]:
data

,id,event_id,down_time,up_time,action_time,activity,text_change,cursor_position,word_count,score,down_event_action_category,up_event_action_category
0,001519c8,1,4526,4557,31,Nonproduction,NoChange,0,0,3.5,Mouse Actions,Mouse Actions
1,001519c8,2,4558,4962,404,Nonproduction,NoChange,0,0,3.5,Mouse Actions,Mouse Actions
2,001519c8,3,106571,106571,0,Nonproduction,NoChange,0,0,3.5,Keyboard Actions,Keyboard Actions
3,001519c8,4,106686,106777,91,Input,q,1,1,3.5,Keyboard Actions,Keyboard Actions
4,001519c8,5,107196,107323,127,Input,q,2,1,3.5,Keyboard Actions,Keyboard Actions
...,...,...,...,...,...,...,...,...,...,...,...,...
8405893,fff05981,3615,2063944,2064440,496,Nonproduction,NoChange,1031,240,4.0,Mouse Actions,Mouse Actions
8405894,fff05981,3616,2064497,2064497,0,Nonproduction,NoChange,1031,240,4.0,Keyboard Actions,Keyboard Actions
8405895,fff05981,3617,2064657,2064765,108,Replace,q => q,1031,240,4.0,Keyboard Actions,Keyboard Actions
8405896,fff05981,3618,2069186,2069259,73,Nonproduction,NoChange,1028,240,4.0,Mouse Actions,Mouse Actions


In [12]:
categorical_cols = ['activity', 'text_change', 'down_event_action_category', 'up_event_action_category']
data_encoded = pd.get_dummies(data, columns=categorical_cols)


: 

In [ ]:
data_encoded.drop(['id', 'event_id'], axis=1, inplace=True)

/Users/ayushlodha/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


In [11]:
# Standardize the remaining numerical features
numeric_cols = data_encoded.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
data_encoded[numeric_cols] = scaler.fit_transform(data_encoded[numeric_cols])

<8405898x4175 sparse matrix of type '<class 'numpy.float64'>'
	with 76886906 stored elements in Compressed Sparse Row format>

In [ ]:
# Perform PCA to reduce dimensions while retaining 95% variance
pca = PCA(n_components=0.95)
principal_components = pca.fit_transform(data_scaled)

# Create a DataFrame with the principal components
pc_columns = [f'PC{i+1}' for i in range(principal_components.shape[1])]
data_pca = pd.DataFrame(principal_components, columns=pc_columns)

# If you want to join back the 'id' for reference
data_pca['id'] = data['id']

print(data_pca)

ValueError: Shape of passed values is (8405898, 1), indices imply (8405898, 4175)